In [5]:
import openai
from dotenv import load_dotenv, find_dotenv
import pprint


# Carrega as variáveis de ambiente
_ = load_dotenv(find_dotenv())

# Cria o cliente da OpenAI
client = OpenAI()


In [2]:
import openai
print(openai.__version__)

1.95.1


### Criar o Assistant

In [20]:
from openai import OpenAI
import time

client = OpenAI()

# 1. Upload do arquivo
uploaded_file = client.files.create(
    file=open("C:/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/Explorando a API da OpenAI.pdf", "rb"),
    purpose="assistants"
)

# 2. Criar assistente com file_search (não use mais 'retrieval')
assistant = client.beta.assistants.create(
    name="Ajuda com Apostila Asimov",
    instructions="Responda perguntas com base na apostila.",
    model="gpt-4-1106-preview",
    tools=[{"type": "file_search"}]
)

# 3. Criar um thread
thread = client.beta.threads.create()

# 4. Adicionar mensagem do usuário
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="O que a apostila diz sobre embeddings?"
)

# 5. Criar o run, agora associando o arquivo via 'tool_resources'
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {
            "file_ids": [uploaded_file.id]
        }
    }
)

# 6. Aguardar finalização da execução
while True:
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run_status.status == "completed":
        break
    elif run_status.status == "failed":
        raise Exception("Execução falhou.")
    time.sleep(1)

# 7. Exibir resposta
messages = client.beta.threads.messages.list(thread_id=thread.id)
for message in reversed(messages.data):
    print(f"{message.role.upper()}: {message.content[0].text.value}")


C:\Users\chris\AppData\Local\Temp\ipykernel_13484\1271609810.py:21: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\chris\AppData\Local\Temp\ipykernel_13484\1271609810.py:24: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\chris\AppData\Local\Temp\ipykernel_13484\1271609810.py:31: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


TypeError: Runs.create() got an unexpected keyword argument 'tool_resources'

In [8]:
vector_store = client.beta.vector_stores.create(
    name = 'Apostilas curso Asimov'
)

AttributeError: 'Beta' object has no attribute 'vector_stores'

In [ ]:
file = ['Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/Explorando a API da OpenAI.pdf']
file_stream = [open(f, 'rb') for f in files]

file_bacth = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id = vector_store.id,
    files=file_stream
)

In [10]:
file = client.files.create(
    file=open('/Users/chris/OneDrive/Documentos/Asimov_Academy/AI_Agents/02_OpenAI_API/supermarket_sales.csv','rb'),
    purpose='assistants'
)


In [11]:
file.id

'file-3BQNZ5SwF2bWBwamSafRtu'

In [13]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

assistant = client.beta.assistants.create(
    name="Analista Financeiro",
    instructions="Você é um analista financeiro de um supermercado. Você deve usar os dados  \
    .csv informados relativos às vendas do supermercado para realizar as suas análises",
    tools=[{"type": "code_interpreter"}],
    tool_resources={'code_interpreter': {'file_ids':[file.id]}},
    model="gpt-4-1106-preview"
)


### Criação de um Thread

In [14]:
thread = client.beta.threads.create()

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\3600338518.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


### Adicionar mensagem à thread criada

In [26]:
texto_mensagem='Qual é o rating médio das vendas do supermercado?'
texto_mensagem='Gere um gráfico de Pizza com o percentual dos meios de pagamento. O arquivo está em formato.csv'

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=texto_mensagem)

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\877700454.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


### Rodar a Thread no assistant

In [27]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Usuário 9999'
)

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\195674185.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


### Aguardar a Thread rodar

In [28]:

import time


while run.status in ['queued','in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


C:\Users\chris\AppData\Local\Temp\ipykernel_11892\641565305.py:6: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [29]:
run.status

'completed'

### Verificar a Resposta

In [30]:
if run.status =='completed':
    messages= client.beta.threads.messages.list(
        thread_id=thread.id
    )

    print(messages)

else:
    print('Erro', run.status)

C:\Users\chris\AppData\Local\Temp\ipykernel_11892\1184797989.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages= client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_bi5WL1r69GDl9wCOdPwqgsFU', assistant_id='asst_AZjFVejnqkbbbxKLNvfDaRTY', attachments=[Attachment(file_id='file-Niag6ufdLtScadqgEpg5hd', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-2GVLJMgoX5mYXQHhAGoN4t', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=275, file_path=FilePath(file_id='file-Niag6ufdLtScadqgEpg5hd'), start_index=211, text='sandbox:/mnt/data/supermarket_payment_distribution_pie_chart.png', type='file_path')], value='O gráfico de pizza com o percentual dos meios de pagamento foi gerado e salvo com sucesso. Você pode baixar o gráfico diretamente através do seguinte link:\n\n[Download do gráfico de pizza dos meios de pagamento](sandbox:/mnt/data/supermarket_payment_distribution_pie_chart.png)'), type='text')], created_at=1753147818, incomplete_at=None, incomplete_details=None, met

In [32]:
print(messages.data[0].content[0])

ImageFileContentBlock(image_file=ImageFile(file_id='file-2GVLJMgoX5mYXQHhAGoN4t', detail=None), type='image_file')


### Analisando os passos do modelo

In [33]:
run_steps= client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
    
)




C:\Users\chris\AppData\Local\Temp\ipykernel_11892\687884520.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps= client.beta.threads.runs.steps.list(


In [34]:
run_steps.data[0]

RunStep(id='step_w2jyWBoTmIN5WwVJd8nnU68q', assistant_id='asst_AZjFVejnqkbbbxKLNvfDaRTY', cancelled_at=None, completed_at=1753147820, created_at=1753147818, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_osYwubScjayAO3mQZDaC6kll', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_bi5WL1r69GDl9wCOdPwqgsFU'), type='message_creation'), thread_id='thread_0I8iIMLnba7XSu460UGHsZsU', type='message_creation', usage=Usage(completion_tokens=68, prompt_tokens=1637, total_tokens=1705, prompt_token_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0}), expires_at=None)

In [44]:
for step in run_steps.data:
    print("=== Step:", step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print("-----------")
            print(tool_call.code_interpreter.input)
            print("-----------")
            print("Result")
            if tool_call.code_interpreter.outputs[0].type== 'logs':
                print(tool_call.code_interpreter.outputs[0].logs)
            if step.step_details.type == 'message_creation':
                message = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=step.step_details.message_creation.message_id
                )
            if message.content[0].type == 'text':
                print(message.content[0].text.value)

            if message.content[0].type == 'image_file':
                file_id = message.content[0].image_file.file_id
                image_data = client.files.content(file_id)
                with open(f'arquivos/{file_id}.png', 'wb') as f:
                    f.write(image_data.read())
                    print(f'Imagem {file_id} salva com sucesso!')
                import matplotlib.pyplot as plt
                import matplotlib.image as mpimg

                img = mpimg.imread(f'arquivos/{file_id}.png')
                fig, ax = plt.subplots()
                ax.set_axis_off()
                ax.imshow(img)
                plt.show()

            

=== Step: message_creation
=== Step: tool_calls
-----------
import matplotlib.pyplot as plt

# Calcular a contagem de cada meio de pagamento
payment_counts = df['Payment'].value_counts()

# Calcular as porcentagens para cada meio de pagamento
payment_percentage = payment_counts / payment_counts.sum() * 100

# Criar o gráfico de pizza
plt.figure(figsize=(8, 8))
plt.pie(payment_percentage, labels=payment_percentage.index, autopct='%1.1f%%', startangle=140)
plt.title('Percentual dos Meios de Pagamento')
plt.axis('equal')  # Para garantir que o gráfico de pizza seja um círculo

# Salvar a figura
output_file_path = '/mnt/data/supermarket_payment_distribution_pie_chart.png'
plt.savefig(output_file_path)

# Mostrar o gráfico
plt.show()

# Retornar o caminho do arquivo onde o gráfico foi salvo
output_file_path
-----------
Result


FileNotFoundError: [Errno 2] No such file or directory: 'arquivos/file-2GVLJMgoX5mYXQHhAGoN4t.png'